In [80]:
import pandas as pd
import pickle5 as pickle
#df.to_pickle('data_with_coord_29_03_21')  
with open('data_with_coord_29_03_21', "rb") as fh:
    df = pickle.load(fh)
df['ResponseTime'] = (df.TimeArrived-df.TimeOfCall).astype('timedelta64[s]')
df['ResponseTime2'] = (df.TimeArrived-df.TimeMobilised).astype('timedelta64[s]')

# ResponseTime : Délai entre l'appel et l'arrivée sur site
rapport de données utiles : ~80%

In [81]:
df = df.sort_values(by='ResponseTime')
print(df.shape, df[df.ResponseTime>0].shape, df[df.ResponseTime>0].shape[0]/df.shape[0])

(1797007, 22) (1435466, 22) 0.7988093535528799


# ResponseTime2 : Délai entre la mobilisation et l'arrivée sur site
rapport de données utiles : ~99.9%

In [82]:
df = df.sort_values(by='ResponseTime2')
print(df[df.ResponseTime2>0].shape[0]/df.shape[0])

0.9980228235059742


# Est-ce que les données manquantes ont un lien avec l'une des catégories ?

In [83]:
# let's binarize
from scipy.stats import chi2_contingency
df['ResponseOk'] = 1
df[df.ResponseTime <0].ResponseOk = 0
res = []
for i in ['DeployedFromStation_Name', 'IncidentGroup', 'DateOfCall', 'CalYear', 'HourOfCall']:
    res.append(chi2_contingency(pd.crosstab(df[i], df.ResponseOk))[1])
print(res)
print("p>0.05 pour toutes, donc aucun lien entre la disponibilité des valeurs et les conditions")

C:\Users\Roman\anaconda3\envs\ptorch\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


[1.0, 1.0, 1.0, 1.0, 1.0]
p>0.05 pour toutes, donc aucun lien entre la disponibilité des valeurs et les conditions


# Distribution délais (minutes)

In [84]:
petites_stats = pd.DataFrame()
petites_stats['Delai_depuis'] = ['Appel','Mobilisation']
petites_stats['minimales'] = [df.ResponseTime[df.ResponseTime>0].min()/60, df.ResponseTime2[df.ResponseTime2>0].min()/60]
petites_stats['maximales'] = [df.ResponseTime[df.ResponseTime>0].max()/60, df.ResponseTime2[df.ResponseTime2>0].max()/60]
petites_stats['moyennes'] = [df.ResponseTime[df.ResponseTime>0].mean()/60, df.ResponseTime2[df.ResponseTime2>0].mean()/60]
petites_stats['std'] = [df.ResponseTime[df.ResponseTime>0].std()/60, df.ResponseTime2[df.ResponseTime2>0].std()/60]

petites_stats

,Delai_depuis,minimales,maximales,moyennes,std
0,Appel,0.016667,90045.566667,28.784874,596.624552
1,Mobilisation,0.050000,1991.650000,6.700305,8.942544


# Est-ce que ces variables sont corrélées ?

In [85]:
from scipy.stats import pearsonr
d1, d2 = df.ResponseTime[df.ResponseTime>0], df.ResponseTime2[df.ResponseTime>0]
r, p = pearsonr(d1,d2)
print('r = ',r, ' P = ', p)
print('laison faible (0.2<r<0.5), mais significative (P<.05)')

r =  0.26154940910673835  P =  0.0
laison faible (0.2<r<0.5), mais significative (P<.05)
